In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from scipy.signal import savgol_filter
import json
from datetime import datetime
import statsmodels.api as sm
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
import os
from helper_functions import *

import requests
import time
from causalimpact import CausalImpact

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import threading
import csv
import gc
from lxml import etree
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, VBox
from IPython.display import display, clear_output
import plotly.io as pio
from matplotlib.lines import Line2D

import dash
from dash import dcc, html

from scipy.stats import pearsonr

- We have in the previous cell downloaded the libraries that will be used to process the dataset. Also, we import most of the relevant datasets in the following cell:

In [ ]:
interventions = load_interventions()
globalmob = load_globalmob()
d = load_aggregated_timeseries()
data, df_code = choose_restrictiveness("All", "Yes")

- The dataset relative to the mobility (apple and global) are large (especially the global one, but we can filter them to keep the data we need. Indeed, both dataset contain the mobility change percentage per day, compared to a day 0, per region/country. In the apple dataset, this change are grouped by walking, driving and transit and in the global dataset, the changes are grouped by reason for displacement (go to the park, to the pharmacy, etc). In both dataset, certain country don't have data for all the displacement types. For example, we don't have the transit mobility for korea in the apple dataset. Also, for certains countries, the period over which we have data is smaller than the normal length of the dataset. For example, in the global dataset, there is missing values for every dates between 2020-05-18 and 2020-07-03. To filter the data we can choose to keep only a few countries or region. We chose to keep the countries that are presented in the "interventions" dataset.

## To what extent can English Wikipedia pages serve as a reliable estimate of the average video game popularity?

- We first compare the mobility between england and the average of all the countries we consider to see if there is any relation between them. As we thought, they are not very similar, or at least there is no obvious trend that they both seem to follow. The English mobility recovers more slowly than the average and is reduced a lot more than the average.

In [ ]:
average_mobility(d, df_code, interventions, globalmob)

- Afterwards we plot the Wikipedia page views related to video games for each country we cosider, as well as the average trend.
- Here we can see the that all of the countries had an increase in video game page views during the lockdown. Some were more impacted than others.
- It is important to take into account that we are plotting the amount of video game page views, while taking into account the total page views for that time, therefore we can see how much more often video games are looked up compared to other pages instead of just seeing the increase in Wikipedia searches, since the Wikipedia usage went up in general during the lockdown.

In [ ]:
plot_percent_pageviews(d, df_code, interventions)

- Here we want to see if the English Wikipedia page views can represent the average of all other countries well in order to be able to use English as an estimate of the worldwide trend.
- At first glance, after having shifted all the lines around 0 in order to be able to compare the trends, English doesn't seem to be a good fit, or at least doesn't seem to be close to the average trend. It actually seems to be on eof the most far off compared to the average, especially during the lockdown as can be seen on the graph below.
- However when we scale down the England trend line (divinding it by 2.5), it can be seen that it follows the average trend very well. This shows that the English wikipedia pages follow the worldwide trend in a more exaggerated fashion.

In [ ]:
plot_normalized_percent_pageviews(d, df_code, interventions)

- Here we look at the smoothened mobility trends to see if England matches any of the three groups better than another, and it fits the semi-restrictive countries the best, which is interesting as the scaled English wikipedia trend above matches the average of the semi-restrictive countries the best. Especially during the lockdown.

In [ ]:
plot_mobility(d, df_code, interventions)

- When looking more specifically at one game, we can see that there are only English Wikipedia searches for Among Us mid-2020. This is due to the fact that the game and it's respective Wikipedia pages only existed in English before youtubers and streamers brought hype to the game.
- First of all we can see how the game's popularity multiplied by 10 during the COVID-19 lockdown.
- Secondly, we can see how smaller video games are exclusively english when first released as developpers don't have the time or resources to put into developping the game in multiple languages. However when games gain popularity, such as Among Us did, developpers put more resources and time into releasing it in more languages in order to attain a larger audience. 
- The English spike was higher, but more short lived than the other languages. This is because when more languages exist on the game, players prefer to play in their own language, and therefore shift what Wikipedia page they search up.
- We can also see that English was a reliable benchmark to see if Among Us would work in another language with the popularity it gained for the developpers. As predicted, it gained popularity in other languages when they became available.
- However if we look at Fall Guys, the numbers of Wikipedia game searches for any language other than English, we cannot say anything as the number of searches are so low (most below 10 searches a day). However if it had enough searches to ignore noise, we could say that it would most likely follow the same trend as the English Wikipedia searches.

In [ ]:
return_specific_game()

## What about specific types of video games

- Since there are many pages in different languages for the same wikipidea item, it is essential to extract the translated version for the languages other than English. 
- Here we use WikiMidea API to extract the current wikipidea pages of the game topics in all languages for further analysis.

In [ ]:
df_topiclinked = pd.read_csv('topics_linked.csv') #import the topic_linked dataset

In [ ]:
div = save_topics_in_chunk(df_topiclinked) # save the datasets into chunks
start_title_crawler_thread(div) # crawl the game titles in different langugages

- After we parse down the game topics in different languages, we then parse down all the pageviews in every game topic in every language in the selected timespan.

In [ ]:
start_dt = '2019100100'
end_dt = '2020123100'
start_pageview_crawler_thread(start_dt, end_dt) #crawl the pageviews for every game title in Wikipidea

In [ ]:
langs =['en','de','fr','it','es','pt','ru','sv','pl','nl','no','da','fi']
thread_num = 8
crawl_uncrawled_pageviews(df_topiclinked,thread_num,langs, start_dt, end_dt)

- Then we parse down the game genres for every game from the official api from the Wikipedia in this notebook.

In [ ]:
start_genre_crawler_thread()

- Then we classify the game topics into main game genres.

In [ ]:
result_df = filter_game_genres(raw_game_filepath='./Milestone3/games.csv')
display(result_df)

- We finish the first round of classification
- We classify the overall game genres into bigger categories, which provides a good outcome in this task.
- Here we input the genres we extract from the Wikipedia and ask chatgpt to conduct the classification on the smaller genres. Since the extracted genres contain enough information for the chatGPT to process, it provides a robust output in this task

In [ ]:
main_genre_df = main_genre_classification('./Milestone3/gpt-classification_raw.csv',result_df)

In [ ]:
# analyze the classified genres.
stats_df = display_main_genre_stats(main_genre_df)
visualize_genres_distribution(stats_df,others_threadshold=100)

### Get the pageviews in different languages and different genres

In [ ]:
visualize_pageviews_in_genre(pageviews_filepath='./Milestone3/pageviews.csv', genres_filepath='./Milestone3/gpt-classification.csv')    

- We can see that some game genres for certain languages have fewer pageviews. But the games on Action, RPG, Miscellaenous have larger pageviews for many languages.
- Then we analyze the potential links between mobility and pageviews in games
- Here we try to analyze whether increased or decreased mobility aligns with heightened interest in specific video game genres, indicating a potential connection between real-world restrictions and virtual entertainment.

In [ ]:
#import the needed dataset
interventions = load_interventions()
applemob = load_applemob()
globalmob = load_globalmob()
pageviews = pd.read_csv('./Milestone3/pageviews.csv')
game_genres = pd.read_csv('./Milestone3/gpt-classification.csv')

baseline = '2020-02-14' #Define it as the baseline for the pageviews
d = load_aggregated_timeseries()

In [ ]:
# Define the languages needed to analyze
data = {
    'France': ['fr', 'FR'],
    'Denmark': ['da', 'DK'],
    'Germany': ['de', 'DE'],
    'Italy': ['it', 'IT'],
    'Netherlands': ['nl', 'NL'],
    'Norway': ['no', 'NO'],
    #'Serbia': ['sr', 'RS'],
    #'Sweden': ['sv', 'SE'],
    #'Korea': ['ko', 'KR'],
    #'Catalonia': ['ca', 'ES'],
    'Finland': ['fi', 'FI'],
    'Japan': ['ja', 'JP']
}
df_code = pd.DataFrame(data)
df_code = df_code.transpose()
df_code.rename(columns = {0:'lang', 1:'state'}, inplace = True)
code_dict = convert_to_code_dict(df_code)

In [ ]:
grouped_df, globalmob = merge_mobility_pageview(globalmob, pageviews, game_genres, code_dict) # merge the mobility dataset with the pageview one

In [ ]:
visualize_mobility_pageviews(globalmob, interventions, df_code)

- The red line indicates how the pageviews change from baseline while the blue line indicates how the mobility changes from baseline. We can clearly see that as soon as the intervention is implemented, the pageviews on game topics instantly increase in major countries, for example France, Germany and Italy. However, it is not so obvious to observe the reversed trend between mobility and pageviews considering Norway.
- After that we tried to look into every country, while considering the pageviews in different game genres.

In [ ]:
visualize_different_language('fr',baseline,interventions, grouped_df,globalmob,code_dict,country_name='France',omit_genre=['Music']) # country with very restrctive intervention during lockdown

In [ ]:
visualize_different_language('de', baseline,interventions,grouped_df,globalmob,code_dict,country_name='Germany',omit_genre=['Music']) #country with restrctive intervention during lockdown

In [ ]:
visualize_different_language('jp', baseline,interventions,grouped_df,globalmob,code_dict,country_name='Japan',omit_genre=['Adult','Adventure','Music']) # country with less restrctive intervention during lockdown

- In the previous studies, we have categorized the lockdown itensity into restrctive, semi-restrctive and unrestrictve, and we now study three countries, one in each of the categories.

### France

- We know that there is avery restrctive implementation in France during the COVID-19 lockdown, and there is a huge amount of time spent on video games, which leads to a constant increase in pageviews of game-related topics. 
- When considering game genres, we can see that during the lockdown, the video games about Action, Adult, Adventure, Horror, Puzzle, RPG, Sports and Strategy have attracted the greatest increase. Note that the Horror games actually remain more popular than pre-lockdown time after the lockdown had been lifted, while other games are back to normal popularity just like before the lockdown.
- We can specifically look into games about Sports and Multiplayer/Online, after society is back to normal, the popularities of these two genres of games actually decline, which is reasonable since people tend to play sports offline and meet offline to hang out and have fun.

### Germany

- We know that Germany is categorized as semi-restrctive during the lockdown, and people tend to enjoy more freedom compared to France during the lockdown. It is not only indicated by the mobility data, but also by the popularity of video games during the lockdown.
- Overall there is a minor increase in the popularity of games, but it is obviously milder than that in France. 
- Regarding different game genres; Miscellanoues, RPG, Stimulation, Sports and Strategy enjoy the most gain from the lockdown, but if you compared it to other same genres in France, it is definitely a smaller gain.

### Japan

- Japan actually enjoys less restrctions during the lockdown than is implemented by the government, so the game industry has almost no benefit received from the lockdown.
- In general, there is almost no sudden increase or decrease by the time the lockdown is implemented. However there are still some minor gains in the game of Multiplayer/Online, Puzzle, Sports and Strategy. 
- Furthermore the popularity seems to last longer than expected after the intervention is over. We believe there is an explanation since the mobility has not changed sharply, the popularity in the games can last longer since people are not "forced" to play games, instead they actually enjoy video games. So there is some evidence that the game industry in Japan actually benefits from covid in a longer timespan.

## Overall relationship between mobility and video games

The dataset relative to the mobility (apple and global) are large (especially the global one, but we can filter them to keep the data we need. Indeed, both dataset contain the mobility change percentage per day, compared to a day 0, per region/country. In the apple dataset, this change are grouped by walking, driving and transit and in the global dataset, the changes are grouped by reason for displacement (go to the park, to the pharmacy, etc). In both dataset, certain country don't have data for all the displacement types. For example, we don't have the transit mobility for korea in the apple dataset. Also, for certains countries, the period over which we have data is smaller than the normal length of the dataset. For example, in the global dataset, there is missing values for every dates between 2020-05-18 and 2020-07-03. To filter the data we can choose to keep only a few countries or region. We chose to keep the countries that are presented in the "interventions" dataset.

In [ ]:
interventions = load_interventions()
djson = load_aggregated_timeseries()

In [ ]:
interventions.loc['ca']['Normalcy'] = '2020-06-25'

## Google Mobility

- In this section, we plotted the evolution of mobility in different country during the Covid-19 period, according to the global mobility dataset.

- The intervention dataset contains important dates in the evolution of covid 19 in several countries. The ISO 639-1 standard is used to characterize the country. Normally, this standard does not characterize a country, but a language. The standard characterizing a country is ISO 3166-1 alpha-2, used in the global_mobility dataset, but it seems that the code in the intervention dataset corresponds to the country associated with the language. For example, in "interventions", we have "fr" which characterizes French, but we'll consider that the "fr" line corresponds to France.

- So, following this logic, we decided to plot the evolution of mobility in France, Denmark, Germany, Italy, the Netherlands, Norway, Serbia, Sweden, South Korea, Catalonia (which is not a country but a region) and Japan.

- To make these plots we used the average of mobility change. In the global_mobility dataset we have the change percentage (compared to a day 0) of different type displacement (retail_and_recreation,grocery_and_pharmacy, ...) and from these, we made an average change per day.
    
- On each plot, the blue curve correspond to this average and the gey ones correspond to the evolution of displacement type.

- On each plot, we can see at most 3 lines. The black dotted line corresponds to the lockdown date, the grey one corresponds to the date of return to normality, the blue one corresponds to the mobility restrictions date. All these dates have been taken from the interventions dataset.


In [ ]:
data, df_code = choose_restrictiveness("All", "No")

In [ ]:
plotmob_inter(globalmob, df_code, interventions)

In [ ]:
plotmob(globalmob, df_code, interventions)

## Lockdown Itensity Google

- In this section we used the global mobility dataset to compare the intensity of lockdowns depending on countries.

- To determine if a lockdown was intense or not, we took the average of the mobility decrease over the lockdown period.

- Also, to be able to compare the curves between them we wanted to plot them on the same graph. To be able to see something we first need to smooth the curves.

- Thanks to the smoothered curves and the average bar plot, we can distinguish 3 types of lockdown: Very restrictive, restrictive, unrestrictive.

In [ ]:
meanmobplot(df_code, interventions, globalmob)

In [ ]:
smoothedmobility(df_code, globalmob)

In [ ]:
aggregatedmobilitysmoothedplot(df_code, globalmob)

In [ ]:
desaggregatedmobilitysmoothed(df_code, globalmob)

## Graph Apple

- In this section, we plotted the evolution of mobility in different country during the Covid-19 period, according to the apple mobility dataset.
- The difference with the google mobility dataset is that the different mobilities are grouped in 3 groups: walking, driving and transit.
- We therefore plotted the 3 differents type of mobility for each country. For 2 countries (Korea and Serbia) we don't have the mobility for transit mobiliyt data. Also, in the interventions dataset we have informations about catalonia and we don't have a location "catalonia". So instead we plotted the mobility data from Barcelona, which is Catalonia's biggest city.

In [ ]:
plotmobapple(applemob, df_code, interventions)

## Json

- In this section we plot the percentage of pageviews related to videogames for each country over time. We can see that there is globally a small increase during Covid lockdown. We also smoothed the curves with a polynomial function.

In [ ]:
pageviewsplot(df_code, interventions, djson)

In [ ]:
aggregatedpageviewsplot(df_code, djson, interventions)

In [ ]:
desaggregatedpageviewsplot(df_code, djson, interventions)

In [ ]:
correlationanalysisplot_false(df_code, globalmob, djson)

In [ ]:
pearsonpageviews_false(df_code, globalmob, djson)

In [ ]:
pearsonpageviews_plot(df_code, globalmob, djson)

In [ ]:
pearsonpageviews_plot_inter(df_code, globalmob, djson)

In [ ]:
medianpercentage_plot(df_code, globalmob, djson, interventions)

In [ ]:
medianpercentage_boxplot(df_code, globalmob, djson, interventions)